In [9]:
import pandas as pd

negative = pd.read_csv("DIJA_REDDIT_top5_neg.csv", engine="python", sep=",", error_bad_lines=False)

In [10]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/Damian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Damian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


negative['cleanText']=negative['Title'].map(lambda s:preprocess(s))

In [12]:
negative.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText,compound,neg,neu,pos
0,473,40613,11976.95996,12087.00977,11936.32031,12044.40039,143670000,12044.40039,0.004989,2.178083,Massive 7.9 earthquake in northeastern Japan. ...,massive earthquake northeastern japan someone ...,0.8360,0.115,0.737,0.148
1,1228,41711,16341.54981,16405.07031,16084.09961,16108.88965,86160000,16108.88965,-0.014149,2.506465,Australia: Coalition Lies to UN in Order to St...,australia coalition lies order strip tasmanian...,-0.8689,0.111,0.801,0.088
2,1482,42080,17972.22070,17972.22070,17785.78906,17849.08008,82560000,17849.08008,-0.007139,2.270520,Irish no-frills airline Ryanair confirmed its ...,irish frills airline ryanair confirmed board a...,-0.3257,0.104,0.790,0.106
3,1551,42179,18139.09961,18139.09961,17966.07031,17966.07031,104980000,17966.07031,-0.009810,2.238120,Alberta passes bill banning corporate and unio...,alberta passes bill banning corporate union do...,-0.9022,0.115,0.807,0.077
4,1586,42229,17401.64063,17481.77930,17341.33984,17408.25000,88550000,17408.25000,0.000330,2.147489,Huge explosion reported in Chinese port city o...,huge explosion reported chinese port city tian...,-0.8934,0.110,0.798,0.092


In [13]:
del negative['compound']
del negative['neg']
del negative['neu']
del negative['pos']

In [14]:
negative.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText
0,473,40613,11976.95996,12087.00977,11936.32031,12044.40039,143670000,12044.40039,0.004989,2.178083,Massive 7.9 earthquake in northeastern Japan. ...,massive earthquake northeastern japan someone ...
1,1228,41711,16341.54981,16405.07031,16084.09961,16108.88965,86160000,16108.88965,-0.014149,2.506465,Australia: Coalition Lies to UN in Order to St...,australia coalition lies order strip tasmanian...
2,1482,42080,17972.22070,17972.22070,17785.78906,17849.08008,82560000,17849.08008,-0.007139,2.270520,Irish no-frills airline Ryanair confirmed its ...,irish frills airline ryanair confirmed board a...
3,1551,42179,18139.09961,18139.09961,17966.07031,17966.07031,104980000,17966.07031,-0.009810,2.238120,Alberta passes bill banning corporate and unio...,alberta passes bill banning corporate union do...
4,1586,42229,17401.64063,17481.77930,17341.33984,17408.25000,88550000,17408.25000,0.000330,2.147489,Huge explosion reported in Chinese port city o...,huge explosion reported chinese port city tian...


In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [16]:
pip install vaderSentiment 

Note: you may need to restart the kernel to use updated packages.


In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [18]:
preprocess = []
analyzer = SentimentIntensityAnalyzer()
for clean in negative['cleanText']:
    vs = analyzer.polarity_scores(clean)
    preprocess.append(vs)

In [19]:
#positive sentiment: compound score >= 0.05
#neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#negative sentiment: compound score <= -0.05

In [20]:
df3 = pd.DataFrame(preprocess)

In [21]:
complete_neg = pd.concat([negative,df3], axis=1)

In [22]:
complete_neg

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText,compound,neg,neu,pos
0,473,40613,11976.95996,12087.00977,11936.32031,12044.40039,143670000,12044.40039,0.004989,2.178083,Massive 7.9 earthquake in northeastern Japan. ...,massive earthquake northeastern japan someone ...,0.8360,0.115,0.737,0.148
1,1228,41711,16341.54981,16405.07031,16084.09961,16108.88965,86160000,16108.88965,-0.014149,2.506465,Australia: Coalition Lies to UN in Order to St...,australia coalition lies order strip tasmanian...,-0.8689,0.111,0.801,0.088
2,1482,42080,17972.22070,17972.22070,17785.78906,17849.08008,82560000,17849.08008,-0.007139,2.270520,Irish no-frills airline Ryanair confirmed its ...,irish frills airline ryanair confirmed board a...,-0.3257,0.104,0.790,0.106
3,1551,42179,18139.09961,18139.09961,17966.07031,17966.07031,104980000,17966.07031,-0.009810,2.238120,Alberta passes bill banning corporate and unio...,alberta passes bill banning corporate union do...,-0.9022,0.115,0.807,0.077
4,1586,42229,17401.64063,17481.77930,17341.33984,17408.25000,88550000,17408.25000,0.000330,2.147489,Huge explosion reported in Chinese port city o...,huge explosion reported chinese port city tian...,-0.8934,0.110,0.798,0.092
5,1734,42445,17249.33984,17379.17969,17204.07031,17325.75977,118710000,17325.75977,0.004303,2.243309,US State Dept declares ISIS is committing geno...,state dept declares isis committing genocide i...,-0.8865,0.118,0.820,0.063


In [24]:
complete_neg.to_csv('top_5_negative.csv')